In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
import joblib
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#load in all of the models
loaded_model_DT = joblib.load('Decison_Tree.sav')
loaded_model_RF = joblib.load('RF_model.sav')
loaded_model_SVC = joblib.load('SVC_model.sav')
loaded_model_KNN = joblib.load('KNN_model.sav')
loaded_model_LR = joblib.load('LR_model.sav')

#create a list that contains all of the models
def get_models():
    models = list()
    models.append((loaded_model_DT))
    models.append((loaded_model_RF))
    models.append((loaded_model_SVC))
    models.append((loaded_model_KNN))
    models.append((loaded_model_LR))
    return models

models = get_models()
print(models)

df = pd.read_excel('nfl.xlsx')

x = df[df.columns[:-1]].to_numpy()
y = df[df.columns[-1]].to_numpy()

n = 80

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

meta_X = list()
for model in models:
    # predict on hold out set
    yhat = model.predict(X_test)
    # reshape predictions into a matrix with one column
    yhat = yhat.reshape(len(yhat), 1)
    # store predictions as input for blending
    meta_X.append(yhat)

meta_X = np.hstack(meta_X)
    
blender = LogisticRegression()
# fit on predictions from base models
blender.fit(meta_X, y_test)

all_models = list()

#crate predictions of individual models
DT = models[0].predict(X_test)
RF = models[1].predict(X_test)
SVC = models[2].predict(X_test)
KNN = models[3].predict(X_test)
LR = models[4].predict(X_test)

counter = 0
model_answers = list()

#make a list for each game containing the invidual model predictions
for i in DT:
    temp = list()
    temp.append(DT[counter])
    temp.append(RF[counter])
    temp.append(SVC[counter])
    temp.append(KNN[counter])
    temp.append(LR[counter])
    model_answers.append(temp)
    counter = counter + 1

#store blender predictions in yhat
yhat = blender.predict(model_answers)

print("y_hat:", yhat)
print("y_test:", y_test)
print(accuracy_score(yhat, y_test))



    

[DecisionTreeClassifier(max_features='log2'), RandomForestClassifier(max_features='log2', min_samples_split=3), SVC(kernel='linear'), KNeighborsClassifier(n_neighbors=7), LogisticRegression(C=0.1, max_iter=100000)]
y_hat: [1 0 0 1 1 0 0 0 1 1 1 1 0 1]
y_test: [1 0 0 1 1 0 0 0 1 1 1 1 0 1]
1.0


In [3]:
df_2021 = pd.read_excel('nfl_2021.xlsx')

#load in 2021 data
x = df_2021[df_2021.columns[:-1]].to_numpy()
y = df_2021[df_2021.columns[-1]].to_numpy()

#classifiers make their individual predictions on the 2021 data
DT_2 = models[0].predict(x)
RF_2 = models[1].predict(x)
SVC_2 = models[2].predict(x)
KNN_2 = models[3].predict(x)
LR_2 = models[4].predict(x)


counter = 0
model_answers = list()

#make a list for each game containing the invidual model predictions
for i in DT_2:
    temp = list()
    temp.append(DT_2[counter])
    temp.append(RF_2[counter])
    temp.append(SVC_2[counter])
    temp.append(KNN_2[counter])
    temp.append(LR_2[counter])
    model_answers.append(temp)
    counter = counter + 1
counter = 0
#store blender predictions in yhat
yhat = blender.predict(model_answers)


print(yhat)
print(y)
print("yhat", accuracy_score(yhat, y))
print("DT", accuracy_score(DT_2, y))
print("RF", accuracy_score(RF_2, y))
print("SVC", accuracy_score(SVC_2, y))
print("KNN", accuracy_score(KNN_2, y))
print("LR", accuracy_score(LR_2, y))


[1 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 0
 1 1 1]
[1 1 1 1 1 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 0 1 0 1 1 0 1
 1 1 1]
yhat 0.7
DT 0.65
RF 0.575
SVC 0.6
KNN 0.65
LR 0.55
